# 리스코프 치환 원칙 (LSP)
리스코프 치환 원칙은 설계 시 안정성을 유지하기 위해 객체 타입이 유지해야하는 일련의 특성  
LSP의 주요 개념은 **어떤 클래스에서든 클라이언트는 특별한 주의를 기울이지 않고도 하위 타입을 사용할 수 있어야 한다는 것**  

> 만약 S가 T의 하위 타입이라면 프로그램을 변경하지 않고 T 타입의 객체를 S 타입의 객체로 치환 가능해야 함

LSP의 규칙에 따르면 하위 클래스는 상위 클래스에서 정의한 계약을 따르도록 디자인 되어야 함

### 도구를 사용해 LSP 문제 검사하기
LSP 문제를 Mypy나 Pylint같은 도구를 사용해 쉽게 검출할 수 있음  

#### 메서드 서명의 잘못된 데이터 타입 검사
Event 클래스의 하위 클래스 중 하나가 호환되지 않는 방식으로 메서드를 재정의하면 Mypy는 어노테이션을 검사하여 이를 확인함

In [2]:
class Event:
    def meets_condition(self, event_data: dict) -> bool:
        return False
    
class LoginEvent(Event):
    def meets_condition(self, event_data: list) -> bool:
        return bool(event_data)

위의 경우 Mypy를 실행하면 다음과 같은 오류 메세지가 표시됨
> error: Argument 1 of "meets_condition" incompatible with supertype "Event"

이와 같은 오류가 나는 이유는 파생 클래스가 부모 클래스에서 정의한 파라미터와 다른 타입을 사용했기 때문임.  
LSP 원칙에 따르면 호출자는 아무런 차이를 느끼지 않고 투명하게 Event 또는 LoginEvent를 사용할 수 있어야 함.  
위의 예제에서 반환 값을 bool이 아닌 다른 값으로 변경해도 동일한 오류 발생

#### Pylint로 호환되지 않는 서명 검사
또 다른 자주 발생하는 LSP 위반 사례 = 계층의 파라미터 타입이 다른 것이 아니라 메서드의 서명 자체가 완전히 다른 경우  

In [3]:
# lsp_1.py
class LogoutEvent(Event):
    def meets_condition(self, event_data: dict, override: bool) -> bool:
        if override:
            return True
        return False

위와 같은 코드에 대해 Pylint는 다음과 같은 정보를 출력
> Parameters differ from overriden 'meets_condition' method (argumentsdiffer)

### 애매한 LSP 위반 사례
자동화된 도구로 LSP를 위반했는 지 여부를 확인하기 어려울 때 코드리뷰를 통해 자세히 코드를 살펴봐야 함  


In [9]:
# lsp_2.py
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data
        
    @staticmethod
    def meets_condition(event_data: dict):
         return False
        
    @staticmethod
    def meets_condition_pre(event_data: dict):
        """
        인터페이스 계약의 사전 조건
        ''event_data'' 파라미터가 적절한 형태인지 유효성 검사
        """
        
        assert isinstance(event_data, dict), f"{event_data!r} is not a dict"
        for moment in ("before", "after"):
            assert moment in event_data, f"{moment} not in {event_data}"
            assert isinstance(event_data[moment], dict)
     
    
class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""
    
class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 0 
            and event_data["after"]["session"] == 1
        )
    
class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 1 
            and event_data["after"]["session"] == 0
        )
            
class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""
    
    def __init__(self, event_data):
        self.event_data = event_data
        
    def identify_event(self):
        Event.meets_condition_pre(self.event_data)
        event_cls = next(
            (
                event_cls
                for event_cls in Event.__subclasses__()
                if event_cls.meets_condition(self.event_data)
            ),
            UnknownEvent,
        )
        return event_cls(self.event_data)

위의 경우 Event에서 키 before와 after가 필수적이고 그 값 또한 사전타입이어야 한다고 명시되어있음  
하위클래스에서 보다 제한적인 파라미터를 요구하는 경우 검사에 통과하지 못함.  
그러나 아래의 트랜잭션 이벤트 클래스는 올바르게 설계되어있음

In [10]:
class TransactionEvent(Event):
    """시스템에서 발생한 이벤트 분류"""
    
    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None

그러나 앞의 LoginEvent와 LogoutEvent 클래스는 before와 after의 "session"이라는 키를 사용하기 때문에 그대로 사용할 수 없음  
이렇게 되면 KeyError가 발생하므로 나머지 클래스를 사용하는 것과 같은 방식으로 클래스를 사용할 수 없음  
따라서 TransactionEvent와 마찬가지로 대괄호 대신 .get()메서드로 수정하여 해결할 수 있음

In [11]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
print(l1.identify_event().__class__.__name__)
l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
print(l2.identify_event().__class__.__name__)
l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
print(l3.identify_event().__class__.__name__)
l4 = SystemMonitor({"before": {}, "after":{"transaction": "Tx001"}})
print(l4.identify_event().__class__.__name__)

LoginEvent
LogoutEvent
UnknownEvent


KeyError: 'session'

In [18]:
# lsp_2.py
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data
        
    @staticmethod
    def meets_condition(event_data: dict):
         return False
        
    @staticmethod
    def meets_condition_pre(event_data: dict):
        """
        인터페이스 계약의 사전 조건
        ''event_data'' 파라미터가 적절한 형태인지 유효성 검사
        """
        
        assert isinstance(event_data, dict), f"{event_data!r} is not a dict"
        for moment in ("before", "after"):
            assert moment in event_data, f"{moment} not in {event_data}"
            assert isinstance(event_data[moment], dict)
     
    
class UnknownEvent(Event):
    """데이터만으로 식별할 수 없는 이벤트"""
    
    
class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"].get("session") == 0
            and event_data["after"].get("session") == 1
        )
    
class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"].get("session") == 1
            and event_data["after"].get("session") == 0
        )
    
class TransactionEvent(Event):
    """시스템에서 발생한 이벤트 분류"""
    
    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None
    
class SystemMonitor:
    """시스템에서 발생한 이벤트 분류"""
    
    def __init__(self, event_data):
        self.event_data = event_data
        
    def identify_event(self):
        Event.meets_condition_pre(self.event_data)
        event_cls = next(
            (
                event_cls
                for event_cls in Event.__subclasses__()
                if event_cls.meets_condition(self.event_data)
            ),
            UnknownEvent,
        )
        return event_cls(self.event_data)

In [19]:
l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
print(l1.identify_event().__class__.__name__)
l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
print(l2.identify_event().__class__.__name__)
l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
print(l3.identify_event().__class__.__name__)
l4 = SystemMonitor({"before": {}, "after":{"transaction": "Tx001"}})
print(l4.identify_event().__class__.__name__)

LoginEvent
LogoutEvent
UnknownEvent
TransactionEvent


### LSP 최종 정리
LSP는 객체 지향 소프트웨어 설계의 핵심이 되는 다형성을 강조하기 때문에 좋은 디자인의 기초가 됨.  
인터페이스의 메서드가 올바른 계층구조를 갖도록 하여 상속된 클래스가 부모 클래스와 다형성을 유지하도록 하는 것  

- 새로운 클래스가 원래의 계약과 호환되지 않는 확장을 하려고 하면 클라이언트와의 계약이 깨져서 결과적으로 그러한 확장이 가능하지 않을 것
- 확장을 가능하게 하려면 수정에 대해 폐쇄되어야 한다는 원칙을 깨야하는데 이는 바람직하지 않은 형태임 

LSP에서 제안하는 방식으로 클래스를 디자인하면 계층을 올바르게 확장하는 데 도움이 됨 -> **LSP가 OCP에 기여**